In [6]:
# 지마켓 베스트셀러 상품 크롤러

#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.

from bs4 import BeautifulSoup
from selenium import webdriver

import time
import sys
import re
import math
import numpy 
import pandas as pd   
import xlwt 
import random
import os

import urllib.request
import urllib


# Step 2. 사용자에게 정보를 입력 받습니다.
print("=" *80)
print("     Gmarket Best Seller 상품 정보 추출하기")
print("=" *80)

query_txt='지마켓'
query_url='https://corners.gmarket.co.kr/Bestsellers'

cnt = int(input('   1.해당 분야에서 크롤링 할 건수는 몇건입니까?(1-200 건 사이 입력): '))
f_dir = input("    2.파일을 저장할 폴더명만 쓰세요(예:c:\\temp\\):")
print("\n")

# Step 3. 저장될 파일위치와 이름을 지정 한 후 크롬 드라이버를 실행하여 페이지를 엽니다
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_dir=f_dir+s+'-'+query_txt
ff_name=f_dir+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'.xls'

s_time = time.time( )

path = "C:\py_temp\chromedriver.exe"
driver = webdriver.Chrome(path)
    
driver.get(query_url)
time.sleep(5)

# ALL 버튼을 눌러 페이지를 엽니다
driver.find_element_by_xpath("""//*[@id="categoryTabG"]/li[1]/a""").click( )
time.sleep(1)

#Step 4. 화면을 스크롤해서 아래로 이동한 후 요청된 데이터를 수집합니다.

def scroll_down(driver):
      
      driver.execute_script("window.scrollBy(0,9300);")
      time.sleep(1)

scroll_down(driver)

# 비트맵 이미지 아이콘을 위한 대체 딕셔너리를 만듭니다
bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)

# 이미지 추출 코드 추가
img_src2=[]   # 이미지 URL 저장변수
file_no = 0

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

reple_result = soup.select('#gBestWrap > div > div:nth-child(5) > div:nth-child(3)')
slist = reple_result[0].find_all('li')

ranking2=[]
title3=[]
s_price2=[]
o_price2=[]
discount2=[]
       
count = 0
    
# 이미지 저장용 폴더 생성하기
img_dir = ff_dir+"\\images"
os.makedirs(img_dir)
os.chdir(img_dir)

for li in slist:
            
            # 이미지 저장하기
            try :
              photo = li.find('div','thumb').find('img')['src']
            except AttributeError :
              continue
            file_no += 1

            urllib.request.urlretrieve(photo,str(file_no)+'.jpg')
            time.sleep(1)

            if cnt == file_no :
              break
            
            f = open(ff_name, 'a',encoding='UTF-8')
            f.write("-----------------------------------------------------"+"\n")

            # 판매순위
            print("-" *70)
            try :
             ranking = li.find('p').get_text().replace("#","")
            except AttributeError :
             ranking = ''
             print(ranking.replace("#",""))
            else :
             print("1.판매순위:",ranking)

             f.write('1.판매순위:'+ ranking + "\n")

            #제품 설명 
            try :
             title1 = li.find('a',class_='itemname').get_text().replace("\n","")
            except AttributeError :
             title1 = ''
             print(title1.replace("\n",""))
             f.write('2.제품소개:'+ title1 + "\n")
            else :
             title2=title1.translate(bmp_map).replace("\n","") 
             print("2.제품소개:", title2.replace("\n",""))

             count += 1
             
             f.write('2.제품소개:'+ title2 + "\n")
            
            # 원래가격
             try :
               o_price = li.find('div',class_='o-price').find('span').find('span').get_text().replace("\n","")
            
             except AttributeError :
               o_price = ''
               
             print("3.원래가격:", o_price.replace("\n",""))
             f.write('3.원래가격:'+ o_price + "\n")
            
             # 판매가격
             try :
               s_price = li.find('div',class_='s-price').find('span').find('span').get_text().replace("\n","")
             except AttributeError :
               s_price = ''
               
             print("4.판매가격:", s_price.replace("\n",""))
             f.write('4.판매가격:'+ s_price + "\n")

            #할인율
             try :
               discount = li.find('em').get_text()
             except AttributeError :
               discount='0%'
               
             print('5.할인율:',discount)
             f.write('5.할인율:'+ discount + "\n")

             print("-" *70)
                          
             f.close( )
              
             time.sleep(0.3)
            
             ranking2.append(ranking)
             title3.append(title2.replace("\n",""))
             o_price2.append(o_price.replace("\n",""))
             s_price2.append(s_price.replace("\n",""))
             discount2.append(discount)             

             if count == cnt :
                break
                          
#Step 5. 검색 결과를 다양한 형태로 저장하기
              
Gmarket_best_seller = pd.DataFrame()
Gmarket_best_seller['판매순위']=ranking2
Gmarket_best_seller['제품소개']=pd.Series(title3)
Gmarket_best_seller['원래가격']=pd.Series(o_price2)
Gmarket_best_seller['판매가격']=pd.Series(s_price2)
Gmarket_best_seller['할인율']=pd.Series(discount2)


# csv 형태로 저장하기
Gmarket_best_seller.to_csv(fc_name,encoding="utf-8-sig",index=True)

# 엑셀 형태로 저장하기
Gmarket_best_seller.to_excel(fx_name ,index=True)

e_time = time.time( )
t_time = e_time - s_time

# txt 파일에 크롤링 요약 정보 저장하기
orig_stdout = sys.stdout
f = open(ff_name, 'a',encoding='UTF-8')
sys.stdout = f


#이미지 삽입하기

import win32com.client as win32   #pywin32 , pypiwin32 설치후 동작
import win32api  #파이썬 프롬프트를 관리자 권한으로 실행해야 에러없음
                     #파이썬 쉘을 관리자 권한으로 실행한 후 불러오기로 이 소스 실행하기
excel = win32.gencache.EnsureDispatch('Excel.Application')
wb = excel.Workbooks.Open(fx_name)
sheet = wb.ActiveSheet
sheet.Columns(3).ColumnWidth = 30   #  이미지 가로 사이즈에 맞게 컬럼 크기 조정
row_cnt = cnt+1
sheet.Rows("2:%s" %row_cnt).RowHeight = 120  #  이미지 세로 사이즈에 맞게 로우 크기 조정

ws = wb.Sheets("Sheet1")
col_name2=[]
file_name2=[]

for a in range(2,cnt+2) :
      col_name='C'+str(a)
      col_name2.append(col_name)

for b in range(1,cnt+1) :
      file_name=img_dir+'\\'+str(b)+'.jpg'
      file_name2.append(file_name)
      
for i in range(0,cnt) :
      rng = ws.Range(col_name2[i])
      image = ws.Shapes.AddPicture(file_name2[i], False, True, rng.Left, rng.Top, 130, 100)
      excel.Visible=True
      excel.ActiveWorkbook.Save()

# Step 6. 요약 정보를 출력하기
print("\n")
print("=" *50)
print("총 소요시간은 %s 초 이며," %t_time)
print("총 저장 건수는 %s 건 입니다 " %count)
print("=" *50)

sys.stdout = orig_stdout
f.close( )

print("\n") 
print("=" *80)
print("1.요청된 총 %s 건의 리뷰 중에서 실제 크롤링 된 리뷰수는 %s 건입니다" %(cnt,count))
print("2.총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("3.파일 저장 완료: txt 파일명 : %s " %ff_name)
print("4.파일 저장 완료: csv 파일명 : %s " %fc_name)
print("5.파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)    

     Gmarket Best Seller 상품 정보 추출하기
   1.해당 분야에서 크롤링 할 건수는 몇건입니까?(1-200 건 사이 입력): 10
    2.파일을 저장할 폴더명만 쓰세요(예:c:\temp\):C:\py_temp\Gmarket\


----------------------------------------------------------------------
1.판매순위: 1
2.제품소개: [좋은순면]유기농 순면 팬티라이너 롱 36매X6팩/생리대 /증정
3.원래가격: 42,000원
4.판매가격: 16,900원
5.할인율: 59%
----------------------------------------------------------------------
----------------------------------------------------------------------
1.판매순위: 2
2.제품소개: [탑텐키즈](신세계타임스퀘어점패션관)[탑텐키즈](정상가 199.900)슈퍼에어 롱 다운 점퍼 2종
3.원래가격: 
4.판매가격: 45,900원
5.할인율: 0%
----------------------------------------------------------------------
----------------------------------------------------------------------
1.판매순위: 3
2.제품소개: [화이트]화이트 입는 오버나이트 대형 8P 2팩/생리대 /증정
3.원래가격: 26,100원
4.판매가격: 10,500원
5.할인율: 59%
----------------------------------------------------------------------
----------------------------------------------------------------------
1.판매순위: 4
2.제품소개: [크리넥스]3겹 순수소프트 화장지 27M 30롤X2팩/휴지 /new
3.원래